# old but gold

In [26]:
import cv2
import json
import os

# Function to draw bounding boxes on an image
def draw_boxes(image, annotation_data):
    for person_data in annotation_data:
        views = person_data["views"]
        for view in views:
            if view['viewNum'] != 1:
                continue
            assert False
            xmin, ymin, xmax, ymax = view["xmin"], view["ymin"], view["xmax"], view["ymax"]
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

# Path to the folder containing frame images
image_folder = "/ssd/r.musaev/deepsort/data/Wildtrack_dataset/Image_subsets/C7/"

# Join the annotated images into a video clip
video_writer = cv2.VideoWriter("/ssd/r.musaev/deepsort/input/Wildtrack/full/camera_6.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 10, (1920, 1080))

# Iterate through all frames
for filename in sorted(os.listdir(image_folder)):
    if filename.endswith(".png"):
        frame_number = filename.split(".")[0]
        image_path = os.path.join(image_folder, filename)
        annotation_path = os.path.join("/ssd/r.musaev/deepsort/data/Wildtrack_dataset/annotations_positions/", f"{frame_number}.json")

        if not os.path.isfile(annotation_path):
            break
        # Load annotation data from JSON file
        with open(annotation_path, "r") as f:
            annotation_data = json.load(f)

        # Load the frame image
        image = cv2.imread(image_path)

        # # Draw bounding boxes on the image
        # draw_boxes(image, annotation_data)

        # Write the annotated frame to the video clip
        video_writer.write(image)

video_writer.release()


In [14]:
import cv2
import numpy as np

def read_video_clip(video_path, max_frame):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = np.ceil(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Determine the maximum number of frames to read
    max_frame = min(max_frame, frame_count)

    frames = []
    current_frame = 0

    while current_frame < max_frame:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        current_frame += 1

    cap.release()
    return frames, fps, width, height

def draw_bbox(frame, bbox):
    # Extract bbox coordinates
    x, y, w, h = bbox
    # Draw bbox on the frame
    cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)

# Read the video clip up to the specified maximum frame
max_frame = 500
video_path = '/ssd/r.musaev/deepsort/data/Wildtrack_dataset/cam2.mp4'
frames, fps, width, height = read_video_clip(video_path, max_frame)

# Path to the annotation file
annotation_file = '/ssd/r.musaev/deepsort/input/Wildtrack/subset/camera_6.txt /ssd/r.musaev/deepsort/input/Wildtrack/subset/camera_5.txt /ssd/r.musaev/deepsort/input/Wildtrack/subset/camera_2.txt /ssd/r.musaev/deepsort/input/Wildtrack/subset/camera_4.txt /ssd/r.musaev/deepsort/input/Wildtrack/subset/camera_3.txt /ssd/r.musaev/deepsort/input/Wildtrack/subset/camera_1.txt /ssd/r.musaev/deepsort/input/Wildtrack/subset/camera_0.txt/camera_1.txt'

# Read annotations
annotations = []
with open(annotation_file, 'r') as f:
    for line in f:
        data = line.strip().split(',')
        frame_num = int(data[0])
        bbox = list(map(int, data[2:6]))
        annotations.append((frame_num, bbox))

# Draw bounding boxes on frames
for frame_num, bbox in annotations:
    if frame_num >= max_frame:
            break
    draw_bbox(frames[frame_num], bbox)

# Join frames into video
output_video_path = '/ssd/r.musaev/deepsort/data/Wildtrack_dataset/cam2_annotated.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

for frame in frames:
    out.write(frame)

out.release()
print("Video saved successfully.")

Video saved successfully.


In [25]:
import json
import os
            
fps_video = 60
fps_anno = 2
FPS_RATIO = 1 #6 #fps_video / fps_anno


def convert_annotations(json_files):
    cameras = {}
    for frame_num, json_file in enumerate(json_files):
        with open(json_file, 'r') as f:
            data = json.load(f)
        
        #frame_num = int(int(json_file.split('/')[-1].split('.')[0]) * FPS_RATIO)
        for person_data in data:
            for view_data in person_data['views']:
                if view_data['xmin'] == -1:
                    continue
                view_num = view_data['viewNum']
                if view_num not in cameras:
                    cameras[view_num] = []
                view_data['xmax'] -= view_data['xmin']
                view_data['ymax'] -= view_data['ymin']
                cameras[view_num].append({
                    'fr_num': frame_num,
                    'personID': person_data['personID'],
                    'boxes_coord': (view_data['xmin'], view_data['ymin'], view_data['xmax'], view_data['ymax'])
                })
    
    return cameras

def write_mot_files(annotations_dir, output_dir):
    json_files = [os.path.join(annotations_dir, f) for f in os.listdir(annotations_dir) if f.endswith('.json')]
    cameras = convert_annotations(json_files)
    
    for view_num, view_data in cameras.items():
        view_data.sort(key=lambda x: x['personID'])  # Sort detections by person ID
        output_file = os.path.join(output_dir, f'camera_{view_num}.txt')
        with open(output_file, 'w') as f:
            for frame_num, person_data in enumerate(view_data, start=1):
                f.write(f"{person_data['fr_num']} {person_data['personID']} {' '.join(map(str, person_data['boxes_coord']))} -1 -1\n")

# Provide the input and output directories
annotations_dir = '/ssd/r.musaev/deepsort/data/Wildtrack_dataset/annotations_positions'
output_dir = '/ssd/r.musaev/deepsort/input/Wildtrack/full'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Convert annotations and write MOT files
write_mot_files(annotations_dir, output_dir)


import pandas as pd
import glob
path = '/ssd/r.musaev/deepsort/input/Wildtrack/full/camera_*.txt'
for path in glob.glob(path):
    df = pd.read_csv(path, names=[
                                'fr_num', 'id', 'bbleft', 'bbtop',
                                'bbox_width', 'bbox_height', 'det_score',
                                'class'], index_col=False, sep=' ')

    df = df.sort_values(by=['fr_num', 'id'])
    df.to_csv(path, index=False, header=None, sep=',')

In [1]:
import numpy as np
from scipy.interpolate import interp1d

# Read annotations from the text file
annotations_file = '/ssd/r.musaev/deepsort/data/Wildtrack_dataset/annotations_positions/camera_0.txt'
annotations_data = np.genfromtxt(annotations_file, delimiter=',', skip_header=1)

# Extract relevant information
frame_nums = annotations_data[:, 0]
ids = annotations_data[:, 1]
bboxes = annotations_data[:, 2:6]  # Extracting bbox_x, bbox_y, bbox_w, bbox_h

# Determine fps and number of frames
fps = 2
num_frames = int(np.max(frame_nums)) + 1

# Perform interpolation for each coordinate separately
interpolated_annotations = np.zeros((num_frames * fps, int(np.max(ids)) + 1, 4))  # Assuming ids are consecutive and start from 0
for i in range(4):  # Iterate over (x, y, width, height)
    interpolated_coords = np.zeros((num_frames * fps, int(np.max(ids)) + 1))
    for idx in np.unique(ids):
        # Extract coordinates for a specific id
        id_mask = ids == idx
        id_frame_nums = frame_nums[id_mask]
        id_bboxes = bboxes[id_mask, i]

        # Create interpolation function for each id
        interp_func = interp1d(id_frame_nums, id_bboxes, kind='linear', axis=0, fill_value="extrapolate")
        # Interpolate coordinates for each id
        interpolated_coords[:, int(idx)] = interp_func(np.arange(num_frames * fps) / fps)

    interpolated_annotations[:, :, i] = interpolated_coords

# Result will be in interpolated_annotations array
# Define the output MOT file name
output_file = annotations_file.replace('.txt', '_interpolated.txt')

# Write interpolated annotations to the MOT file
with open(output_file, 'w') as f:
    # Write header
    f.write("frame_num,id,bbox_x,bbox_y,bbox_w,bbox_h,-1,-1\n")
    
    # Write interpolated annotations
    for frame_num in range(num_frames * fps):
        for idx in range(interpolated_annotations.shape[1]):
            bbox_x, bbox_y, bbox_w, bbox_h = interpolated_annotations[frame_num, idx]
            f.write(f"{frame_num},{idx},{bbox_x},{bbox_y},{bbox_w},{bbox_h},-1,-1\n")

print(f"Interpolated annotations saved to {output_file}")


/ssd/r.musaev/python/miniconda3/envs/mcmt/lib/python3.9/site-packages/scipy/interpolate/_interpolate.py:710: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


In [8]:
import os
import pandas as pd
path = '/ssd/r.musaev/deepsort/data/Wildtrack_dataset/annotations_positions/camera_0_interpolated.txt'
pred = pd.read_csv(path, names=[
    'fr_num', 'id', 'bbleft', 'bbtop',
    'bbox_width', 'bbox_height', 'det_score',
    'class'],
                    index_col=False)

/tmp/ipykernel_2924630/3398986690.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_2924630/3398986690.py:4: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  pred = pd.read_csv(path, names=[


In [ ]:
# CUDA_VISIBLE_DEVICES=0 python3 /ssd/r.musaev/mmdetection/demo/mot_demo.py /ssd/r.musaev/deepsort/data/mvmhat_1/1/images bytetrack_yolox_x_8xb4-amp-80e_crowdhuman-mot17halftrain_test-mot17halfval.py --checkpoint rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth --out /ssd/r.musaev/deepsort/output_mmtracking 

In [ ]:
# python demo/demo_mot_vis.py \
#     configs/vis/masktrack_rcnn/masktrack_rcnn_r50_fpn_12e_youtubevis2019.py \
#     --input /ssd/r.musaev/deepsort/data/Wildtrack_dataset/cam1_clip01.mp4 \
#     --checkpoint checkpoints/masktrack_rcnn_r50_fpn_12e_youtubevis2019_20211022_194830-6ca6b91e.pth \
#     --output /ssd/r.musaev/deepsort/output_mmtracking.mp4 \
#     --show

In [ ]:
# from torchvision.transforms import ToTensor
# import cv2
# import torch
# from PIL import Image
# import numpy as np

# path = '/ssd/r.musaev/deepsort/data/Wildtrack_dataset/Image_subsets/C2/00000000.png'
# frame = np.asarray(Image.open(path))

# resized_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# # Convert frame to tensor and send it to device (cpu or cuda)
# frame_tensor = ToTensor()(resized_frame)

# from mmdet.apis import init_detector, inference_detector

# config_file = 'rtmdet_tiny_8xb32-300e_coco.py'
# checkpoint_file = 'rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'
# model = init_detector(config_file, checkpoint_file, device='cpu')  # or device='cuda:0'
# detecions = inference_detector(model, frame_tensor)

In [ ]:
    # # build the model from a config file and a checkpoint file
    # model = init_track_model(
    #     args.config,
    #     args.checkpoint,
    #     args.detector,
    #     args.reid,
    #     device=args.device)

    # # build the visualizer
    # visualizer = VISUALIZERS.build(model.cfg.visualizer)
    # visualizer.dataset_meta = model.dataset_meta

    # prog_bar = mmengine.ProgressBar(len(imgs))
    # # test and show/save the images
    # for i, img in enumerate(imgs):
    #     if isinstance(img, str):
    #         img_path = osp.join(args.inputs, img)
    #         img = mmcv.imread(img_path)
    #     # result [TrackDataSample]
    #     result = inference_mot(model, img, frame_id=i, video_len=len(imgs))
    #     if args.out is not None:
    #         if in_video or out_video:
    #             out_file = osp.join(out_path, f'{i:06d}.jpg')
    #         else:
    #             out_file = osp.join(out_path, img.rsplit(os.sep, 1)[-1])
    #     else:
    #         out_file = None

In [ ]:
# import pandas as pd

# def get_gt_bboxes(gt_camera, frame_count):
#     """look up for gt_bboxes on frame_count frame"""

#     gt_bboxes = gt_camera[gt_camera.fr_num == frame_count].iloc[:, 2:6].values
#     gt_ids = gt_camera[gt_camera.fr_num == frame_count].iloc[:, 1].values

#     return gt_bboxes, gt_ids

# path = '/ssd/r.musaev/deepsort/input/AIC22_Track1_MTMC_Tracking/S02c006.txt'
# pred = pd.read_csv(path, names=[
#     'fr_num', 'id', 'bbleft', 'bbtop',
#     'bbox_width', 'bbox_height', 'det_score',
#     'class', 'visibility', 'stuff'],
#                     index_col=False)

# frame_count = 0
# gt_bboxes, gt_ids = get_gt_bboxes(pred, frame_count)
# len(gt_bboxes)

In [ ]:
from PIL import Image
import os
import numpy as np

def loadroi(imf):
    """Read the ROI image for a given camera.

    Params
    ------
    cid : int
        Camera ID whose ROI image should be retrieved.
    Returns
    -------
    im : numpy.ndarray
        Image stored as a 2-d ndarray.
    """


    # img = Image.open(imf, mode='r')
    img = np.asarray(Image.open(imf))

    return img


imf = '/ssd/r.musaev/deepsort/data/AIC22_Track1_MTMC_Tracking/validation/S02/c006/roi.jpg'
img = loadroi(imf)
img.shape

In [ ]:
def isROIOutlier(row, roi, height, width):
    """Check whether item stored in row is outside the region of interest.
    
    Params
    ------
    row : pandas.Series
        Row of data including, at minimum, the 'X', 'Y', 'Width', and 'Height' columns.
    roi : numpy.ndarray
        ROI image for the camera with the same id as in row['CameraId'].
    height : int
        ROI image height.
    width : int
        ROI image width.
    Returns
    -------
    bool
        Return True if image is an outlier.
    """
    xmin = row['X']
    ymin = row['Y']
    xmax = row['X'] + row['Width']
    ymax = row['Y'] + row['Height']

    if xmin >= 0 and xmin < width:
        if ymin >= 0 and ymin < height and roi[ymin, xmin] < 255:
            return True
        if ymax >= 0 and ymax < height and roi[ymax, xmin] < 255:
            return True
    if xmax >= 0 and xmax < width:
        if ymin >= 0 and ymin < height and roi[ymin, xmax] < 255:
            return True
        if ymax >= 0 and ymax < height and roi[ymax, xmax] < 255:
            return True
    return False

In [ ]:
CUDA_VISIBLE_DEVICES= python3 eval.py --input_folder /ssd/r.musaev/deepsort/input/Wildtrack/full --mode fasterrcnn_resnet50_fpn_v2 --embedder torchreid  --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter

In [ ]:
CUDA_VISIBLE_DEVICES=2 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_resnet50_fpn_v2 --embedder torchreid --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter --nms_max_overlap 1.0

In [ ]:
; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_resnet50_fpn_v2 --embedder clip_RN50x4; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_resnet50_fpn_v2 --embedder clip_RN50x16; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_resnet50_fpn_v2 --embedder clip_ViT-B/32; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_resnet50_fpn_v2 --embedder clip_ViT-B/16;

# detectors grid search

In [ ]:
CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_resnet50_fpn --embedder mobilenet --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_mobilenet_v3_large_fpn --embedder mobilenet --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_mobilenet_v3_large_320_fpn --embedder mobilenet --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fcos_resnet50_fpn --embedder mobilenet --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model ssd300_vgg16 --embedder mobilenet --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model ssdlite320_mobilenet_v3_large --embedder mobilenet --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model retinanet_resnet50_fpn --embedder mobilenet --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model retinanet_resnet50_fpn_v2 --embedder mobilenet --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter

In [ ]:
CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_resnet50_fpn --embedder mobilenet --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter ; CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_resnet50_fpn --embedder torchreid --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter

In [ ]:
CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/data/mvmhat_1 --model fasterrcnn_mobilenet_v3_large_fpn --embedder mobilenet --global_reid_model_wts osnet_ibn_x1_0_imagenet 

In [ ]:
CUDA_VISIBLE_DEVICES=1 python3 eval.py --input_folder /ssd/r.musaev/deepsort/input/AIC22_Track1_MTMC_Tracking  --mode fasterrcnn_resnet50_fpn_v2  --embedder torchreid  --global_reid_model_wts osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter --cls 2 3

In [ ]:
import glob
import shutil
for src in sorted(glob.glob('/ssd/r.musaev/deepsort/data/AIC22_Track1_MTMC_Tracking/validation/S02/**/*.avi')):
    dst = '/ssd/r.musaev/deepsort/input/AIC22_Track1_MTMC_Tracking/S02'
    dst += src.split('S02/')[1].replace('/', '_').replace('avi','mp4')
    dst = dst.replace('_vdo', '')

    shutil.copyfile(src, dst)


In [ ]:
import glob
import shutil
for src in sorted(glob.glob('/ssd/r.musaev/deepsort/data/AIC22_Track1_MTMC_Tracking/validation/S02/*/gt/gt.txt')):
    print(src)
    dst = '/ssd/r.musaev/deepsort/input/AIC22_Track1_MTMC_Tracking/S02'
    dst += src.split('S02/')[1].replace('/', '_').replace('avi','mp4')
    dst = dst.replace('mp4', 'txt')
    dst = dst.replace('_gt_gt','')
    shutil.copyfile(src, dst)


In [ ]:
def normalize(vector):
    return vector / np.linalg.norm(vector)

import pandas as pd 
import numpy as np
import ast

gt_file = '/ssd/r.musaev/deepsort/outputs/52/S02c007_retinanet_resnet50_fpn_v2_torchreid_osnet_ibn_x1_0_imagenet.txt'
df = pd.read_csv(gt_file, names=[
                            'fr_num', 'id', 'bbleft', 'bbtop',
                            'bbox_width', 'bbox_height', 'det_score',
                            'class', 'visibility', 'stuff', 'desc_reid'], index_col=False)
                        
df.desc_reid = df.desc_reid.apply(lambda x: np.array(ast.literal_eval(x)))


In [ ]:
df

In [ ]:
import motmetrics as mm

def evaluate_tracker(preds, gt):

    # Create MOTAccumulator object  
    acc = mm.MOTAccumulator() 

    # Update with ground truth and predictions
    for frame in range(1, len(gt) + 1):
        gt_boxes = [[b[2], b[3], b[4], b[5]] for b in gt[gt.fr_num == frame].values]
        hp_boxes = [[b[2], b[3], b[4], b[5]] for b in preds[preds.fr_num == frame].values]
        print(gt_boxes)
        print(hp_boxes)
        
        if gt_boxes and hp_boxes:
            dists = mm.distances.iou_matrix(gt_boxes, hp_boxes, max_iou=0.5)
        
        else:
            dists = np.zeros((0,0))

        print(type(gt_boxes))

        acc.update(
            gt_boxes,  
            hp_boxes, 
            dists,
            frameid=frame)
        
        print(acc.events) 
            
    # Get summary metrics  
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['num_frames', 'mota', 'motp'], name='acc')
    
    return summary  

# Usage
summary = evaluate_tracker(df, df)
print(f"MOTA: {summary['mota']}, MOTP: {summary['motp']}")

In [ ]:
import pandas as pd 

VIDEO_PATH = '/ssd/r.musaev/deepsort/data/mvmhat_1/mvmhat_1_1.mp4'
gt = pd.read_csv(VIDEO_PATH.replace('mp4', 'txt'), names=[
    'fr_num', 'id', 'bbleft', 'bbtop',
    'bbox_width', 'bbox_height', 'det_score',
    'class', 'visibility', 'stuff',],
                    index_col=False)

In [ ]:

def scale_gt_bbox(gt_bboxes, original_hw, target_hw):
    """"applies scaling of gt-bboxes"""
    
    gt_bboxes.bbleft = gt_bboxes.bbleft.apply(lambda x: int(x * target_hw[1]/original_hw[1]))
    gt_bboxes.bbtop = gt_bboxes.bbtop.apply(lambda x: int(x * target_hw[0]/original_hw[0]))
    gt_bboxes.bbox_width = gt_bboxes.bbox_width.apply(lambda x: int(x * target_hw[1]/original_hw[1]))
    gt_bboxes.bbox_height = gt_bboxes.bbox_height.apply(lambda x: int(x * target_hw[0]/original_hw[0]))
    
    return gt_bboxes

VIDEO_PATH = '/ssd/r.musaev/deepsort/data/mvmhat_1/mvmhat_1_1.mp4'
gt = pd.read_csv(VIDEO_PATH.replace('mp4', 'txt'), names=[
    'fr_num', 'id', 'bbleft', 'bbtop',
    'bbox_width', 'bbox_height', 'det_score',
    'class', 'visibility', 'stuff',],
                    index_col=False)

original_hw = (1520, 2704)
target_hw = (574, 1024)
gt = scale_gt_bbox(gt, original_hw, target_hw)



In [ ]:
bboxes = gt[gt.fr_num == 234].iloc[:, 2:6].values
ids = gt[gt.fr_num == 234].iloc[:, 1].values


# nuscenes

In [ ]:
def bcubed_eval_bcu(hyp_obj_pairs):
    """
    Evaluate clustering using B-cubed metrics: precision, recall, and F1-score.

    Params:
        hyp_obj_pairs (list): List of tuples where:
            (i_pred_cluster, j_gt_label) - correct assignment
            (i_pred_cluster, -1) - false positive
            (-1, j_gt_label) - false negative

    Returns:
        tuple: (precision, recall, fscore)
    """
    # Initialize dictionaries to store counts
    cluster_counts = {}
    label_counts = {}
    intersection_counts = {}

    # Count assignments and occurrences
    for cluster, label in hyp_obj_pairs:
        if cluster != -1:
            cluster_counts[cluster] = cluster_counts.get(cluster, 0) + 1
        if label != -1:
            label_counts[label] = label_counts.get(label, 0) + 1
        if cluster != -1 and label != -1:
            intersection_counts[(cluster, label)] = intersection_counts.get((cluster, label), 0) + 1

    print(cluster_counts)
    print(label_counts)
    # Calculate precision and recall for each cluster
    precisions = {}
    recalls = {}
    for cluster, label in hyp_obj_pairs:
        if cluster == -1 or label == -1:
            continue
        tp = intersection_counts.get((cluster, label), 0)
        precision = tp / cluster_counts[cluster]
        recall = tp / label_counts[label]
        precisions[cluster] = precisions.get(cluster, 0) + precision
        recalls[label] = recalls.get(label, 0) + recall

    # Calculate overall precision and recall
    overall_precision = sum(precisions.values()) / len(cluster_counts)
    overall_recall = sum(recalls.values()) / len(label_counts)

    # Calculate F1-score
    fscore = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

    return overall_precision, overall_recall, fscore

# Example usage
hyp_obj_pairs = [
    (1, 1), (1, 1), (1, 2), (2, 1), (3, -1), (-1, 2), (-1, 3)
]

precision, recall, fscore = bcubed_eval_bcu(hyp_obj_pairs)
print("Precision:", precision)
print("Recall:", recall)
print("F-score:", fscore)


In [ ]:
# import bcubed

# def bcubed_eval(hyp_obj_pairs):
#     """
#     Evaluate clustering using B-cubed metrics: precision and recall.

#     Params:
#         hyp_obj_pairs (list): List of tuples where:
#             (i_pred_cluster, j_gt_label) - correct assignment
#             (i_pred_cluster, -1) - false positive
#             (-1, j_gt_label) - false negative

#     Returns:
#         tuple: (mean_precision, mean_recall)
#     """
#     # Initialize dictionaries for predicted clusters and ground truth labels
#     pred_clusters = {}
#     gt_labels = {}

#     # Populate dictionaries from the input list
#     for cluster, label in hyp_obj_pairs:
#         if cluster != -1:
#             pred_clusters.setdefault(cluster, set()).add(label)
#         if label != -1:
#             gt_labels.setdefault(label, set()).add(cluster)

#     # Calculate precision and recall using bcubed library
#     precision = bcubed.precision(pred_clusters, gt_labels)
#     recall = bcubed.recall(pred_clusters, gt_labels)

#     return precision, recall

# # Example usage
# hyp_obj_pairs = [
#     (1, "A"), (1, "B"), (2, "A"), (3, -1), (-1, "B"), (-1, "C")
# ]

# precision, recall = bcubed_eval(hyp_obj_pairs)
# print("Precision:", precision)
# print("Recall:", recall)


In [ ]:
import torch
torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)

In [ ]:
import json
import os
from collections import defaultdict
import cv2
import numpy as np

CUT_FRAMES = 5000

# Function to read the JSON file
def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Function to group frames by camera
def group_frames_by_camera(frames):
    camera_groups = defaultdict(list)
    for frame in frames:
        camera_name = frame['filename'].split('/')[-2]
        camera_groups[camera_name].append(frame)
    return camera_groups

# Function to sort frames by timestamp
def sort_frames_by_timestamp(frames):
    return sorted(frames, key=lambda x: x['timestamp'])

# Function to join frames into a video
def join_frames_into_video(frames, output_video_path):
    filepath = os.path.join('/ssd/r.musaev/deepsort/data/nuimages/', frames[0]['filename'])
    frame = cv2.imread(filepath)
    height, width, _ = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    for frame_info in frames[:CUT_FRAMES]:
        filepath = os.path.join('/ssd/r.musaev/deepsort/data/nuimages/', frame_info['filename'])

        if not os.path.isfile(filepath):
            frame = np.zeros_like(frame)
        else:
            frame = cv2.imread(filepath)
            height, width, _ = frame.shape
            out = cv2.VideoWriter(output_video_path, fourcc, 2, (width, height))    
    
        frame = cv2.imread(filepath)
        out.write(frame)
    
    out.release()


In [ ]:
# Read the JSON file
data = read_json_file('/ssd/r.musaev/deepsort/data/nuimages/v1.0-val/sample_data.json')

# # Group frames by camera
camera_groups = group_frames_by_camera(data)

# # Process each camera group
for camera_name, frames in camera_groups.items():
    # Sort frames by timestamp
    sorted_frames = sort_frames_by_timestamp(frames)
    
    # Join frames into a video
    output_video_path = f'mini_{camera_name}_video.mp4'
    join_frames_into_video(sorted_frames, output_video_path)
    print(f'Video saved: {output_video_path}')

In [ ]:
# # Read the JSON file
# data = read_json_file('/ssd/r.musaev/deepsort/data/nuimages/v1.0-mini/sample_data.json')

# # # Group frames by camera
# camera_groups = group_frames_by_camera(data)

# # # Process each camera group
# for camera_name, frames in camera_groups.items():
#     # Sort frames by timestamp
#     sorted_frames = sort_frames_by_timestamp(frames)
    
#     # Join frames into a video
#     output_video_path = f'mini_{camera_name}_video.mp4'
#     join_frames_into_video(sorted_frames, output_video_path)
#     print(f'Video saved: {output_video_path}')


In [ ]:
import pandas as pd
pd.read_csv('/ssd/r.musaev/deepsort/outputs/54/metrics_mot.csv')

In [ ]:
import glob
camera_name = 'CAM_FRONT_LEFT'
frame_pathes = sorted(glob.glob(f'/ssd/r.musaev/deepsort/data/nuimages/sweeps/{camera_name}/*'), key=lambda x: x.split('_')[-1])

filepath = frame_pathes[0]
frame = cv2.imread(filepath)
height, width, _ = frame.shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path = f'mini_{camera_name}_video.mp4'
out = cv2.VideoWriter(output_video_path, fourcc, 2.0, (width, height))

for frame_path in frame_pathes[:1000]:
    frame = cv2.imread(frame_path)
    out.write(frame)

out.release()
output_video_path

len(frame_pathes)

In [ ]:
# import json
# import os
# from collections import defaultdict
# import cv2
# import numpy as np
# from pycocotools import mask as mask_utils

# # Function to read the JSON file
# def read_json_file(file_path):
#     with open(file_path, 'r') as f:
#         data = json.load(f)
#     return data

# # Function to group frames by camera
# def group_frames_by_camera(frames):
#     camera_groups = defaultdict(list)
#     for frame in frames:
#         camera_name = frame['filename'].split('/')[-2]
#         camera_groups[camera_name].append(frame)
#     return camera_groups

# # Function to sort frames by timestamp
# def sort_frames_by_timestamp(frames):
#     return sorted(frames, key=lambda x: x['timestamp'])

# # filter out class
# def get_category_tokens(category_data, target_category):
#     tokens = []    
#     for category in category_data:
#         if target_category in category['name']:
#             tokens.append(category['token'])
#     return tokens

# # Function to join frames into a video with annotations
# def join_frames_into_annotated_video(frames, annotations, category_meta, output_video_path):
#     filepath = os.path.join('/ssd/r.musaev/deepsort/data/nuimages/', frames[0]['filename'])

#     frame = cv2.imread(filepath)
#     height, width, _ = frame.shape
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

#     # define category:
#     category_tokens = get_category_tokens(category_meta, 'vehicle')
    
#     for frame_info in frames:
#         filepath = os.path.join('/ssd/r.musaev/deepsort/data/nuimages/', frame_info['filename'])
#         frame = cv2.imread(filepath)
        
#         # Retrieve annotations for the current frame
#         frame_annotations = [ann for ann in annotations if ann['sample_data_token'] == frame_info['token'] and ann['category_token'] in category_tokens]
        
#         # Draw bounding boxes on the frame
#         for ann in frame_annotations:
#             bbox = ann['bbox']
#             cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
        
#         out.write(frame)
    
#     out.release()


# # Read the JSON files
# frame_data = read_json_file('/ssd/r.musaev/deepsort/data/nuimages/v1.0-mini/sample_data.json')
# annotation_data = read_json_file('/ssd/r.musaev/deepsort/data/nuimages/v1.0-mini/object_ann.json')
# category_data = read_json_file('/ssd/r.musaev/deepsort/data/nuimages/v1.0-mini/category.json')

# # Group frames by camera
# camera_groups = group_frames_by_camera(frame_data)

# # Process each camera group
# for camera_name, frames in camera_groups.items():
#     # Sort frames by timestamp
#     sorted_frames = sort_frames_by_timestamp(frames)
    
#     # Join frames into an annotated video
#     output_video_path = f'{camera_name}_annotated_video.mp4'
#     join_frames_into_annotated_video(sorted_frames, annotation_data, category_data, output_video_path)
#     print(f'Annotated video saved: {output_video_path}')

In [ ]:
import os
from tqdm import tqdm
TOTAL_FRAMES = 1000

def save_annotations_in_mot_format(frames, annotations, category_meta, output_mot_file):
    with open(output_mot_file, 'w') as f:
        # Write MOT header
        # f.write("frame_num, object_id, x_min, y_min, width, height\n")
        
        # define category:
        category_tokens = get_category_tokens(category_meta, 'vehicle')
        
        for frame_info in frames[:TOTAL_FRAMES]:
            # Retrieve frame number
            frame_num = frame_info['timestamp']  # Modify this according to your frame numbering
            
            # Retrieve annotations for the current frame and filter only vehicles
            frame_annotations = [ann for ann in annotations if ann['sample_data_token'] == frame_info['token'] and ann['category_token'] in category_tokens]
            
            # Write annotations to the MOT file
            for ann in frame_annotations:
                bbox = ann['bbox']
                object_id = ann['token']  # Use annotation token as object ID
                x_min, y_min, width, height = bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]
                line = f"{frame_num},{object_id},{x_min},{y_min},{width},{height}\n"
                f.write(line)

# Modify the function to include saving annotations in MOT format
def join_frames_save_annotation(frames, annotations, category_meta, output_video_path, draw_boxes=False):
    filepath = os.path.join('/ssd/r.musaev/deepsort/data/nuimages/', frames[0]['filename'])
    frame = cv2.imread(filepath)
    height, width, _ = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps=20, frameSize=(width, height))

    # Define category:
    category_tokens = get_category_tokens(category_meta, 'vehicle')
    
    # Specify the output MOT file path
    output_mot_file = output_video_path.replace('.mp4', '.txt')

    for frame_info in tqdm(frames[:TOTAL_FRAMES]):
        filepath = os.path.join('/ssd/r.musaev/deepsort/data/nuimages/', frame_info['filename'])
        if not os.path.isfile(filepath):
            continue
        frame = cv2.imread(filepath)
        
        if draw_boxes:
            # Retrieve annotations for the current frame
            frame_annotations = [ann for ann in annotations if ann['sample_data_token'] == frame_info['token'] and ann['category_token'] in category_tokens]
            
            # Draw bounding boxes on the frame
            for ann in frame_annotations:
                bbox = ann['bbox']
                cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
            
        out.write(frame)
    
    # Save annotations in MOT format
    save_annotations_in_mot_format(frames, annotations, category_meta, output_mot_file)

In [ ]:
# frame_annotations = [ann for ann in annotation_data if ann['sample_data_token'] == frame_data[0]['token']]
# frame_annotations


In [ ]:
# sorted_frames[6]

In [ ]:
# Read the JSON files
frame_data = read_json_file('/ssd/r.musaev/deepsort/data/nuimages/v1.0-val/sample_data.json')
annotation_data = read_json_file('/ssd/r.musaev/deepsort/data/nuimages/v1.0-val/object_ann.json')
category_data = read_json_file('/ssd/r.musaev/deepsort/data/nuimages/v1.0-val/category.json')

# Group frames by camera
camera_groups = group_frames_by_camera(frame_data)

# Process each camera group
for camera_name, frames in camera_groups.items():
    # Sort frames by timestamp
    sorted_frames = sort_frames_by_timestamp(frames)
    
    # Join frames into an annotated video
    output_video_path = f'/ssd/r.musaev/deepsort/input/nuimages/v1.0-test/{camera_name}.mp4'
    join_frames_save_annotation(sorted_frames, annotation_data, category_data, output_video_path)
    print(f'Annotated video saved: {output_video_path}')

In [ ]:
import pandas as pd 
import numpy as np
import ast

gt_file = '/ssd/r.musaev/deepsort/outputs/57/S02c006_retinanet_resnet50_fpn_v2_torchreid_osnet_ibn_x1_0_imagenet.txt'
df = pd.read_csv(gt_file, names=[
                            'fr_num', 'id', 'bbleft', 'bbtop',
                            'bbox_width', 'bbox_height', 'det_score',
                            'class', 'visibility', 'stuff', 'desc_reid'], index_col=False)
df

In [ ]:
import pandas as pd 
import numpy as np
import ast

gt_file = '/ssd/r.musaev/deepsort/input/nuimages/v1.0-test/CAM_BACK_LEFT.txt'
df = pd.read_csv(gt_file, names=[
                            'fr_num', 'id', 'bbleft', 'bbtop',
                            'bbox_width', 'bbox_height', 'det_score',
                            'class', 'visibility', 'stuff', 'desc_reid'], index_col=False)
df.fr_num.unique()                


In [ ]:
import numpy as np
import ast

gt_file = '/ssd/r.musaev/deepsort/outputs/57/video4_retinanet_resnet50_fpn_v2_torchreid_osnet_ibn_x1_0_msmt17_combineall_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter.txt'
df = pd.read_csv(gt_file, names=[
                            'fr_num', 'id', 'bbleft', 'bbtop',
                            'bbox_width', 'bbox_height', 'det_score',
                            'class', 'visibility', 'stuff', 'desc_reid'], index_col=False)
df['desc_reid'] = df['desc_reid'].apply(lambda x: np.array(ast.literal_eval(x)))

df_mcmt = pd.DataFrame(
    columns=[
        'fr_num', 'id', 'bbleft', 'bbtop',
        'bbox_width', 'bbox_height', 'det_score', 'class'
    ]
)


num_camera = 0

df['camera'] = num_camera

tracks = [f"{x}_c{num_camera}" for x in df['id']]

df['id'] = df['id'].apply(lambda x: f"{x}_c{num_camera}")

df_sbst = df[['camera', 'id', 'bbleft', 'bbtop',
                'bbox_width', 'bbox_height', 'det_score', 'class']]

df_mcmt = pd.concat([df_mcmt, df_sbst])


In [ ]:
sample_data = [
    (1, 2),  # True positive
    (1, 1),  # True positive
    (2, 1),  # True positive
    (2, 2),  # True positive
    (3, -1), # False positive
    (4, -1), # False positive
    (5, -1), # False positive
    (-1, 3), # False negative
    (-1, 4), # False negative
    (-1, 5)  # False negative
]

# Initialize dictionaries to store clusters and labels
clusters = {}
labels = {}

# Fill in clusters and labels dictionaries
for cluster, label in sample_data:
    if cluster != -1:
        if cluster in clusters:
            clusters[cluster].append(label)
        else:
            clusters[cluster] = [label]
    
    if label != -1:
        if label in labels:
            labels[label].append(cluster)
        else:
            labels[label] = [cluster]


In [ ]:
# Calculate precision
precision = 0.0
total_pairs = 0
for cluster, label_list in clusters.items():
    for label in label_list:
        correct = label_list.count(label)
        total_pairs += correct
        precision += correct / len(label_list)

precision /= total_pairs

# Calculate recall
recall = 0.0
total_pairs = 0
for label, cluster_list in labels.items():
    for cluster in cluster_list:
        correct = cluster_list.count(cluster)
        total_pairs += correct
        recall += correct / len(cluster_list)

recall /= total_pairs

print("Precision:", precision)
print("Recall:", recall)


In [ ]:
bcubed_eval(sample_data)


In [ ]:
def get_roi(path):
    img = Image.open(path, mode='r')
    img.load()
    if img.size[0] > img.size[1]:
        img = img.transpose(Image.TRANSPOSE)
        
    im = np.asarray(img, dtype="uint8")
    if im.shape[0] > im.shape[1]:
        im = im.T

    return img

In [ ]:
from tqdm import tqdm
def bcubed_eval(hyp_obj_pairs):
    """cluster evaluation function
    Based on https://link.springer.com/article/10.1007/s10791-008-9066-8

    Params
    ------
    hyp_obj_pairs: list
        list of tuples where:
            (i_pred_cluster, j_gt_label) - correct assignment
            (i_pred_cluster, -1) - fp
            (-1, j_gt_label) - fn

    Returns
    ------
    (mean_p, mean_r): tuple
        mean (precision, recall) per threshold
    """

    AVERAGE_PRECISION = 0.0
    AVERAGE_RECALL = 0.0
    AVERAGE_FP = 0.0
    recall_items = 0
    precision_items = 0
    used_pairs = set()

    hyp_obj_pairs = sorted(hyp_obj_pairs)
    for pair in tqdm(hyp_obj_pairs):
        if pair in used_pairs:
            continue

        tp = 0
        same_cluster = 0
        same_label = 0

        if pair[0] == pair[1] == -1:
            print('\n\n\n\n\n')
            print('pair[0] == pair[0] == -1')
            print('\n\n\n\n\n')

        if pair[1] == -1:  # FP
            AVERAGE_PRECISION += 0
            precision_items += 1
            AVERAGE_FP += 1
            continue

        if pair[0] == -1:  # FN
            AVERAGE_RECALL += 0
            recall_items += 1
            continue

        for item in hyp_obj_pairs:
            if item[0] == pair[0]:
                same_cluster += 1
                if item[1] == pair[1]:
                    tp += 1

            if item[1] == pair[1]:
                same_label += 1

        p = tp / same_cluster
        assert p <= 1
        AVERAGE_PRECISION += p * tp
        precision_items += tp

        r = tp / same_label
        assert r <= 1
        AVERAGE_RECALL += r * tp
        recall_items += tp
        used_pairs.add(pair)

    mean_p = AVERAGE_PRECISION / precision_items
    mean_r = AVERAGE_RECALL / recall_items
    mean_fp_rate = AVERAGE_FP / precision_items
    print("mean_fp_rate", mean_fp_rate, "1-mean_p", 1-mean_p, "mean_p", mean_p)
    return (mean_p, mean_r, mean_fp_rate)

# draw gt boxes

In [ ]:
import os
import cv2
import pandas as pd

video_path = '/ssd/r.musaev/deepsort/input/AIC22_Track1_MTMC_Tracking/S02c007.mp4'
out_dir = '/ssd/r.musaev/deepsort/input/'

# Video stream
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_fps = int(cap.get(5))
frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
save_name = video_path.split(os.path.sep)[-1].split('.')[0]

# Define codec and create VideoWriter object
out = cv2.VideoWriter(
    f"{out_dir}/{save_name}_annotated.mp4",
    cv2.VideoWriter_fourcc(*'mp4v'), frame_fps,
    (frame_width, frame_height)
)

frame_count = 0  # To count total frames
total_fps = 0  # To get the final frames per second

gt_camera = pd.read_csv(video_path.replace('mp4', 'txt'), names=[
    'fr_num', 'id', 'bbleft', 'bbtop',
    'bbox_width', 'bbox_height', 'det_score',
    'class', 'visibility', 'stuff'],
                    index_col=False)

while cap.isOpened():
    # Read a frame
    ret, frame = cap.read()
    if ret:
        frame_count += 1

        # Get ground truth bounding boxes for the current frame
        gt_bboxes = gt_camera[(gt_camera['fr_num'] == frame_count)]
        
        # Draw ground truth bounding boxes
        for _, bbox in gt_bboxes.iterrows():
            left = int(bbox['bbleft'])
            top = int(bbox['bbtop'])
            width = int(bbox['bbox_width'])
            height = int(bbox['bbox_height'])
            cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)

        # Write the annotated frame to the output video
        out.write(frame)

    else:
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import os

def create_video_from_images(image_folder, output_video_path, fps=30):
    # Get the list of images in the folder
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Be sure to use the lower case
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

# Specify input image folder and output video path
image_folder = '/ssd/r.musaev/deepsort/data/Wildtrack_dataset/Image_subsets/C1'
output_video_path = '/ssd/r.musaev/deepsort/data/Wildtrack_dataset/Image_subsets/C1.mp4'

# Create the video from images
create_video_from_images(image_folder, output_video_path)


# AIC22_Track1_MTMC_Tracking

In [ ]:
import pandas as pd

df = pd.read_csv('/ssd/r.musaev/deepsort/data/AIC22_Track1_MTMC_Tracking/eval/ground_truth_validation.txt', names=[
    'camera', 'fr_num', 'id', 'bbleft', 'bbtop',
    'bbox_width', 'bbox_height', 'det_score',
    'class',], index_col=False, sep=' ')


df = df[df.camera.apply(lambda x: x in [6, 7, 8, 9])]
df.to_csv('/ssd/r.musaev/deepsort/data/AIC22_Track1_MTMC_Tracking/eval/ground_truth_validation_S02.txt', header=None, index=False)

In [ ]:
import mmcv
from mmcv import Config
from mmtracking import init_dist, inference_detector, show_result_pyplot, inference_track
from mmtracking.models import build_tracker


: 

In [ ]:
import mmcv
from mmcv import Config
from mmtracking import init_dist, inference_detector, show_result_pyplot, inference_track
from mmtracking.models import build_tracker

# Step 1: Prepare your data
# Make sure your data is organized appropriately with necessary annotations.

# Step 2: Install mmtracking
# Install mmtracking and its dependencies using pip.

# Step 3: Configure the tracker
# Load the tracker configuration file
config_file = 'path/to/your/tracker_config.py'
cfg = Config.fromfile(config_file)

# Build the tracker model
model = build_tracker(cfg.model)

# Load model checkpoint
checkpoint_file = 'path/to/your/checkpoint.pth'
model.load_state_dict(mmcv.load(checkpoint_file)['state_dict'])

# Step 4: Run the tracker
# Initialize distributed computing environment if necessary
init_dist('cpu', port=29500)

# Load your video data
video_file = 'path/to/your/video.mp4'
video = mmcv.VideoReader(video_file)

# Run inference on the video
results = []
for frame in video:
    result = inference_track(model, frame)
    results.append(result)

# Visualize the tracking results
for frame, result in zip(video, results):
    show_result_pyplot(model, frame, result)
